### 1. `csv` 파일 자동 분할

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# min_samples_splitSV 파일 로드
data = pd.read_csv("C:/py/FakeNewsProject/News_Dataset_Half.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/py/FakeNewsProject/News_Dataset_Half.csv'

In [21]:
# train-test 분할 (80:20)
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [22]:
# 분할된 데이터 저장
train_data.to_csv("train.csv", index=False, encoding="utf-8")
test_data.to_csv("test.csv", index=False, encoding="utf-8")

### 1. 분리

In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
import joblib

In [24]:
# 데이터 로드 및 정제
data = pd.read_csv("C:/py/FakeNewsProject/News_Dataset_Half.csv")  # 데이터 파일 경로
data = data.dropna(subset=['Content', 'Label'])  # 결측값 제거
data['Content'] = data['Content'].str.lower().str.replace(r'[^\\w\\s]', '', regex=True)

# 데이터 분리
X = data['Content']
y = data['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=8000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# 레이블 추출
y_train = train_data['Label']
y_test = test_data['Label']

### 2. 모델 학습

In [25]:
# 랜덤 포레스트 모델 학습
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42, class_weight='balanced')
rf_clf.fit(X_train_tfidf, y_train)

# 예측 및 평가
y_pred = rf_clf.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("Random Forest Accuracy:", accuracy)
print("Classification Report:\n", classification_report(y_test, y_pred))

# 모델 저장
joblib.dump(rf_clf, "random_forest_model.pkl")

Random Forest Accuracy: 0.3076923076923077
Classification Report:
               precision    recall  f1-score   support

           0       0.30      0.97      0.46       404
           1       0.63      0.02      0.04       935

    accuracy                           0.31      1339
   macro avg       0.47      0.50      0.25      1339
weighted avg       0.53      0.31      0.17      1339



['random_forest_model.pkl']

In [26]:
import pandas as pd
import joblib

# 파일 경로 설정
model_path = 'logistic_model.pkl'  # 학습된 모델 파일
vectorizer_path = 'tfidf_vectorizer.pkl'  # 학습된 벡터라이저 파일
sample_file_path = 'Sample_News_Dataset.csv'  # 실험용 데이터 파일

In [27]:
# 1. 샘플 데이터 로드
sample_data = pd.read_csv(sample_file_path, encoding='euc-kr')

# 2. 학습된 모델과 벡터라이저 로드
try:
    model = joblib.load(model_path)
    vectorizer = joblib.load(vectorizer_path)
except FileNotFoundError as e:
    raise Exception("모델 또는 벡터라이저 파일이 누락되었습니다. 올바른 파일을 제공해주세요.") from e

c:\python310\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\python310\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.6.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\python310\lib\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.6.1 when using version 1.6.0. This might lead to breaking code or invalid results. Use

In [28]:
# 3. 샘플 데이터 벡터화
sample_X = vectorizer.transform(sample_data['Content'])

# 4. 예측 수행
predictions = model.predict(sample_X)

# 5. 결과 추가
sample_data['Prediction'] = predictions
sample_data['Prediction'] = sample_data['Prediction'].map({0: "Fake", 1: "Real"})

In [29]:
# 6. 결과 출력
print(sample_data[['Title', 'Prediction']])

# 결과를 min_samples_splitSV로 저장하려면
# sample_data.to_csv("predictions.csv", index=False, encoding="euc-kr")

                           Title Prediction
0  알코올이 코로나에 좋다…가짜뉴스에 최소 800명 사망       Fake
1     알코올이 코로나에 좋다'는 가짜뉴스로 인한 피해       Fake
